In [6]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn import tree
import graphviz 
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils import shuffle

def get_netpara_tree(tree,feature_names,class_num):
    threshold = tree.tree_.threshold
    feature = tree.tree_.feature
    value = tree.tree_.value
    left = tree.tree_.children_left
    right = tree.tree_.children_right
    n=len(threshold)
    m=len(feature_names)
    k=int((n+1)/2)
    a = np.zeros((k-1,m))
    b = np.zeros((k,k-1))
    c = np.zeros((k,class_num))
    j=0
    node_ixs=np.zeros(n)
    node_ixs=node_ixs-1
    for i in range(n):
        if threshold[i] != -2:
            a[j,feature[i]]=threshold[i]
            node_ixs[i]=j
            j=j+1
    def find(x,right,left):
        for i in range(len(right)):
            if right[i]==x:
                return i,-1
        for i in range(len(left)):
            if left[i]==x:
                return i,1
    j=0
    for i in range(n):
        if right[i]== -1:
            x,y= find(i,right,left)
            b[j,int(node_ixs[x])]=y
            while(x!=0):
                x,y= find(x,right,left)
                b[j,int(node_ixs[x])]=y
            c[j]=value[i]
            j=j+1
    return a,b,c

def change_parameters1(fcn,a_c):
    a,b=fcn.named_parameters()
    n = len(a)
    for i in range(len(a[1])):
        for j in range(len(a[1][0])):
            if a_c[i][j]==0:
                (a[1].data)[i][j] = 0
            else:
                (a[1].data)[i][j] = 1
                (b[1].data)[i] = -a_c[i][j]
#     print (a)
#     print (b)
    
def change_parameters2(fcn,c_c):
    a,b=fcn.named_parameters()
    n = len(a)
    for i in range(len(a[1])):
        for j in range(len(a[1][0])):
            (a[1].data)[i,j] = c_c[j][i]
    for i in range(len(b[1])):
        (b[1].data)[i] = 0
    
class Tree_net(nn.Module):
    def __init__(self,a,b,c,class_num):
        super(Tree_net, self).__init__()
        self.fcn1 = nn.Linear(len(a[0]), len(a))
        self.fcn2 = nn.Linear(len(b), class_num)
        self.b=b 
        self.a=a
        self.c=c
        change_parameters1(self.fcn1,self.a)
        change_parameters2(self.fcn2,self.c)
    def forward(self, input):
        a,b=self.fcn1.named_parameters()
        batch_size = len(input)
        x = self.fcn1(input)
        m = len(self.b)
        x = x*10
        x = torch.sigmoid(x)
        x_c = Variable(torch.ones(batch_size,m))
        for bs in range(batch_size):
            for i in range(m):
                for j in range(len(self.b[i])):
                    if self.b[i][j] == 1:
                        x_c[bs,i] = x_c[bs,i].clone() *(1-x[bs,j]) 
                    if self.b[i][j] == -1:
                        x_c[bs,i] = x_c[bs,i].clone() *x[bs,j]
        output=self.fcn2(x_c)
        return output


class random_forest_net(nn.Module):
    def __init__(self,random_forest,feature_names,class_num):
        super(random_forest_net, self).__init__()
        self.n_estimators=len(random_forest.estimators_)
        self.forest_net=nn.ModuleList()
        for i in range(self.n_estimators):
            tree = random_forest.estimators_[i]
            a,b,c=get_netpara_tree(tree,feature_names,class_num)
            tree_net=Tree_net(a,b,c,class_num)
            (self.forest_net).append(tree_net)
            
    def forward(self, input):
        x = Variable(torch.zeros(class_num))
        for i in range(self.n_estimators):
            x=(self.forest_net[i])(input)/self.n_estimators
        return x

def get_accuracy(y_predict,y_true):
    m=0.
    for i in range(len(y_predict)):
        if y_predict[i]== y_true[i]:
            m=m+1
    return m/len(y_predict)

def test_tree_net(rfc_net,X_test,y_test):
    n=len(X_test)
    m=int(n/100)
    accuracy_all = 0
    for i in range(m):
        x=torch.FloatTensor(X_test[0+100*i:100+100*i])
        out = rfc_net(x)
        y_predict=(torch.max(F.softmax(out), 1)[1]).numpy()
        y_true=y_test[0+100*i:100+100*i]
        accuracy_all = accuracy_all+get_accuracy(y_predict,y_true)
    accuracy_all=accuracy_all/m
    print ("net_result:")
    print (accuracy_all)

def train(rfc_net,X_train,y_train,X_test,y_test):
    for i in range(100):
        loss_all = 0
        X_train,y_train = shuffle(X_train,y_train)
        for j in range(30):
            learning_rate = 0.1 # If you set this too high, it might explode. If too low, it might not learn 
            criterion=nn.CrossEntropyLoss()
            optimizer = optim.SGD(rfc_net.parameters(), lr=learning_rate, momentum=0.9)
            inputs=torch.FloatTensor(X_train[0+j*100:100+j*100])
            lable=torch.LongTensor(y_train[0+j*100:100+j*100])
            output= rfc_net(inputs)
        #     print (output)
            loss = criterion(output, lable)
            loss.backward()
            optimizer.step()
            loss_all = loss+loss_all
        loss_all = loss_all/30
        print (loss_all)
        if i%5==0:
#             test_tree_net(rfc_net,X_train,y_train)
            test_tree_net(rfc_net,X_test,y_test)
    return rfc_net

In [3]:
X_train = np.loadtxt("data/Adam_output_train.txt")
X_test = np.loadtxt("data/Adam_output_test.txt")
Y_train = np.loadtxt("data/yAdamtrain.txt")
Y_test = np.loadtxt("data/yAdamtest.txt")
Y = np.ones(len(Y_train))
for i in range(len(Y)):
    for j in range(4):
        if Y_train[i,j] == 1:
            Y[i] = j
ytest =  np.ones(len(Y_test))
for i in range(len(ytest)):
    for j in range(4):
        if Y_test[i,j] == 1:
            ytest[i] = j

In [15]:
rfc = RandomForestClassifier(n_estimators=100,max_depth=20)
rfc.fit(X_train, Y)
y_predict=rfc.predict(X_test)
print(get_accuracy(y_predict,ytest))
y_predict=rfc.predict(X_train)
get_accuracy(y_predict,Y)

0.6032388663967612


1.0

In [2]:
X_test = np.loadtxt("data/SGD_output_test.txt")
X_train = np.loadtxt("data/SGD_output_train.txt")
Y_train = np.loadtxt("data/ySGDtrain.txt")
Y_test = np.loadtxt("data/ySGDtest.txt")
Y = np.ones(len(Y_train))
for i in range(len(Y)):
    for j in range(4):
        if Y_train[i,j] == 1:
            Y[i] = j
ytest =  np.ones(len(Y_test))
for i in range(len(ytest)):
    for j in range(4):
        if Y_test[i,j] == 1:
            ytest[i] = j

In [12]:
rfc = RandomForestClassifier(n_estimators=80,max_depth=19)
rfc.fit(X_train, Y)
y_predict=rfc.predict(X_test)
get_accuracy(y_predict,ytest)
print(get_accuracy(y_predict,ytest))
y_predict=rfc.predict(X_train)
get_accuracy(y_predict,Y)

0.5678137651821862


0.9997467071935157

In [5]:
for i in range (50):
    rfc = RandomForestClassifier(n_estimators=2*i+2,max_depth=15)
    rfc.fit(X_train, Y)
    y_predict=rfc.predict(X_test)
    get_accuracy(y_predict,ytest)
    print(get_accuracy(y_predict,ytest))
#         y_predict=rfc.predict(X_train)
#         print(get_accuracy(y_predict,Y))

0.4807692307692308
0.5242914979757085
0.49291497975708504
0.5101214574898786
0.5303643724696356
0.5465587044534413
0.5566801619433198
0.5546558704453441
0.5465587044534413
0.5566801619433198
0.5718623481781376
0.555668016194332
0.562753036437247
0.5668016194331984
0.5708502024291497
0.562753036437247
0.5688259109311741
0.5718623481781376
0.5678137651821862
0.5789473684210527
0.5718623481781376
0.5657894736842105
0.5728744939271255
0.5657894736842105
0.5759109311740891
0.5718623481781376
0.5546558704453441
0.562753036437247
0.5789473684210527
0.5819838056680162
0.5840080971659919
0.5657894736842105
0.5728744939271255
0.5587044534412956
0.5688259109311741
0.5769230769230769
0.5789473684210527
0.5688259109311741
0.5840080971659919
0.5728744939271255
0.5748987854251012
0.5748987854251012
0.5748987854251012
0.5759109311740891
0.5728744939271255
0.5657894736842105
0.5718623481781376
0.5769230769230769
0.5809716599190283
0.5738866396761133
